In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
from biosppy.signals import ecg
import scipy.signal as ss
from scipy.stats import skew, kurtosis
from scipy.fftpack import fft
from neurokit2 import ecg_invert
from tqdm import tqdm
from sklearn.utils import resample
import pandas as pd
from imblearn.over_sampling import SMOTE

In [2]:
# Load data
def load_data(train_path, test_path):
    train = pd.read_csv(train_path, index_col="id")
    test = pd.read_csv(test_path, index_col="id")
    return train, test


def process_signal(ecg, signal_cols_len, wavelet='bior4.4', level=8):
    ecg_signal = ecg
    coeffs = pywt.wavedec(ecg_signal, wavelet, level=level)
    coeffs[0] = np.zeros_like(coeffs[0])
    coeffs[1] = np.zeros_like(coeffs[1])
    coeffs[2] = np.zeros_like(coeffs[2])
    denoised_signal = pywt.waverec(coeffs, wavelet)

    corrected_valid_signal, _ = ecg_invert(denoised_signal[:len(ecg_signal)], sampling_rate=300, show=False)

    std = corrected_valid_signal.std()
    ret = (corrected_valid_signal - corrected_valid_signal.mean()) / std if std > 0 else corrected_valid_signal
    return np.pad(ret, (0, signal_cols_len - len(ret)), 'constant', constant_values=(np.nan))


def preprocess_signals(data):
    signal_cols = [col for col in data.columns if col.startswith('x')]
    l = len(signal_cols)
    # Apply processing to each row (signal)
    data[signal_cols] = data[signal_cols].apply(
        lambda row: pd.Series(process_signal(row.dropna().values, l)), axis=1
    )
    return data




In [3]:
# Load train and test data
train_path = "data/train.csv"
test_path = "data/test.csv"
train, test = load_data(train_path, test_path)


In [4]:
# Preprocess signals  (already done if inversion file)
processed_train = preprocess_signals(train)
processed_test = preprocess_signals(test)

# Verify
print(f"Train shape: {train.shape}, Test shape: {test.shape}")

Train shape: (5117, 17808), Test shape: (3411, 17807)


In [5]:
import matplotlib.pyplot as plt


def plot_signal(signal, sampling_rate=300, title="ECG Signal"):
    """
    Plot an ECG signal.

    Parameters:
    - signal: 1D array-like, the ECG signal to plot.
    - sampling_rate: Integer, sampling rate of the signal (default=300 Hz).
    - title: String, title of the plot.
    """
    # Create a time axis based on the sampling rate
    time = np.linspace(0, len(signal) / sampling_rate, len(signal))

    # Plot the signal
    plt.figure(figsize=(30, 6))
    plt.plot(time, signal, label="ECG Signal")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


In [5]:
def extract_stats(signal):
    signal = signal[~np.isnan(signal)]
    if len(signal) == 0:
        return [0, 0, 0, 0, 0]
    if len(signal) == 1:
        return [signal[0], 0, signal[0], signal[0], signal[0]]
    return [np.mean(signal), np.std(signal), np.median(signal), min(signal), max(signal)]

In [46]:
def extract_features(signal, sampling_rate=300):
    #try:
    features = []
    # ECG Processing
    _, filtered, rpeaks, _, _, _, heart_rate = ecg(signal, sampling_rate=sampling_rate, show=False)
    filtered = filtered[~np.isnan(filtered)]
    rpeaks = rpeaks[~np.isnan(rpeaks)]
    # RR Intervals
    rr_intervals = np.diff(rpeaks) / sampling_rate

    features.append(extract_stats(rr_intervals))

    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    sdnn = np.std(rr_intervals)
    cvsd = rmssd / sdnn if sdnn > 0 else 0
    pnn50 = np.sum(np.abs(np.diff(rr_intervals)) > 0.05) / len(rr_intervals)

    features.append([rmssd, sdnn, cvsd, pnn50])

    delineation = ecg_delineate(filtered, rpeaks, sampling_rate)[1]
    q_peaks = delineation.get("ECG_Q_Peaks", [])
    s_peaks = delineation.get("ECG_S_Peaks", [])
    p_peaks = delineation.get("ECG_P_Peaks", [])
    t_peaks = delineation.get("ECG_T_Peaks", [])

    # Clean peaks
    valid_q = ~np.isnan(q_peaks)
    valid_s = ~np.isnan(s_peaks)
    valid_p = ~np.isnan(p_peaks)
    valid_t = ~np.isnan(t_peaks)
    q_peaks, s_peaks = np.array(q_peaks)[valid_q].astype(int), np.array(s_peaks)[valid_s].astype(int)
    p_peaks, t_peaks = np.array(p_peaks)[valid_p].astype(int), np.array(t_peaks)[valid_t].astype(int)

    pr_intervals_size = min(len(rpeaks), len(p_peaks))
    qt_intervals_size = min(len(q_peaks), len(t_peaks))
    ps_intervals_size = min(len(p_peaks), len(s_peaks))

    pr_intervals = rpeaks[:pr_intervals_size] - p_peaks[:pr_intervals_size]
    qt_intervals = t_peaks[:qt_intervals_size] - q_peaks[:qt_intervals_size]
    ps_intervals = s_peaks[:ps_intervals_size] - p_peaks[:ps_intervals_size]

    features.append(extract_stats(pr_intervals))
    features.append(extract_stats(qt_intervals))
    features.append(extract_stats(ps_intervals))

    # Amplitude Features

    features.append(extract_stats(filtered[rpeaks]))
    features.append(extract_stats(filtered[p_peaks]))
    features.append(extract_stats(filtered[s_peaks]))
    features.append(extract_stats(filtered[t_peaks]))
    features.append(extract_stats(filtered[q_peaks]))

    # Non Linear

    features.append(pyent.sample_entropy(signal, 2, 0.2 * np.nanstd(signal))[0] if len(signal) > 0 else 0)
    features.append(np.nanstd(rr_intervals) / np.sqrt(2) if len(rr_intervals) > 0 else 0)
    features.append(np.sqrt(2) * np.nanstd(rr_intervals) if len(rr_intervals) > 0 else 0)

    artifact_ratio = np.sum(np.abs(signal) > 3 * np.nanstd(signal)) / max(len(signal), 1)
    features.append(artifact_ratio)

    # Frequency-Domain Features
    freq = np.fft.rfftfreq(len(signal), 1 / sampling_rate)
    spectrum = np.abs(np.fft.rfft(signal))
    low_freq_band = (freq >= 0.04) & (freq <= 0.15)
    high_freq_band = (freq >= 0.15) & (freq <= 0.4)

    features.append(
        [freq[np.argmax(spectrum)], np.nanmean(spectrum), np.nanstd(spectrum), np.sum(spectrum[low_freq_band]),
         np.sum(spectrum[high_freq_band]),
         np.sum(spectrum[low_freq_band]) / (np.sum(spectrum[high_freq_band]) + 1e-10)])
    # Autocorrelation Features
    autocorr = np.correlate(signal, signal, mode="full") / len(signal)
    autocorr = autocorr[autocorr.size // 2:]
    features.append(extract_stats(autocorr))

    features.append([np.ptp(signal), kurtosis(signal), skew(signal)])

    # Wavelet Transform Features
    coeffs = pywt.wavedec(signal, 'db4', level=5)
    for i, coeff in enumerate(coeffs[1:], start=1):
        features.append(np.sum(np.square(coeff)))

    features_flat = []
    for f in features:
        if isinstance(f, list):
            features_flat.extend(f)
        else:
            features_flat.append(f)

    f = {index: value for index, value in enumerate(features_flat)}
    return f

#except Exception as e:
#    print(e)
#    return {index: value for index, value in enumerate([0] * 63)}


In [9]:
from joblib import Parallel, delayed
from tqdm import tqdm


def process_row(row, signal_cols, sampling_rate):
    """
    Process a single row to extract features.

    Parameters:
        row (pd.Series): Single row of the DataFrame.
        signal_cols (list): List of signal column names.
        sampling_rate (int): Sampling rate for feature extraction.

    Returns:
        dict: Extracted features for the row.
    """
    signal = row[signal_cols].dropna().to_numpy(dtype="float32")
    features = extract_features(signal, sampling_rate=sampling_rate)  # Use your existing `extract_features` function
    features["id"] = row.name
    if "y" in row:
        features["label"] = row["y"]
    return features


def extract_features_from_df_parallel(data, sampling_rate=300, n_jobs=-1):
    """
    Extract features from a DataFrame in parallel.

    Parameters:
        data (pd.DataFrame): DataFrame containing signal data.
        sampling_rate (int): Sampling rate for feature extraction.
        n_jobs (int): Number of CPU cores to use (-1 for all cores).

    Returns:
        pd.DataFrame: DataFrame of extracted features.
    """
    signal_cols = [col for col in data.columns if col.startswith('x')]

    # Use Parallel for efficient processing
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_row)(row, signal_cols, sampling_rate)
        for _, row in tqdm(data.iterrows(), total=len(data), desc="Extracting Features in Parallel")
    )

    return pd.DataFrame(results)




In [47]:
# Extract features for train and test
train_features = extract_features_from_df_parallel(processed_train, sampling_rate=300)
test_features = extract_features_from_df_parallel(processed_test, sampling_rate=300)

Extracting Features in Parallel: 100%|██████████| 3411/3411 [12:40<00:00,  4.49it/s]


# Model Training (Random Params) - 80% Train, 20% Validation

In [35]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score, confusion_matrix, classification_report

# Prepare data
train_features = pd.read_csv("features/final_train_features.csv")
test_features = pd.read_csv("features/final_test_features.csv")

X = train_features.fillna(0).drop(columns=["id", "label"])  # Features
y = train_features["label"]  # Target

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define individual classifiers
svm_pipeline = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=3.9495, probability=True, random_state=42, class_weight='balanced')
)
gb_pipeline = make_pipeline(
    StandardScaler(),
    HistGradientBoostingClassifier(
        learning_rate=0.013,
        max_iter=212,
        random_state=42,
        class_weight='balanced',
        l2_regularization=0.2
    )
)
xgb_pipeline = make_pipeline(
    StandardScaler(),
    XGBClassifier(
        learning_rate=0.2,
        max_depth=7,
        eval_metric="mlogloss",
        random_state=42
    )
)

# Create ensemble
ensemble = VotingClassifier(
    estimators=[
        ('svm', svm_pipeline),
        ('gb', gb_pipeline),
        ('xgb', xgb_pipeline)
    ],
    voting='soft'
)

# Define pipeline with SMOTE and ensemble
pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('classifier', ensemble)
])

# Perform Stratified K-Fold Cross-Validation
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print("Performing cross-validation...")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=stratified_kfold, scoring="f1_micro", n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# Train and evaluate on validation set
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
f1 = f1_score(y_val, y_pred, average="micro")
print(f"Validation F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))
print("Classification Report:")
print(classification_report(y_val, y_pred))

Performing cross-validation...
Cross-Validation F1 Scores: [0.83028083 0.82905983 0.82051282 0.80562347 0.83251834]
Mean CV F1 Score: 0.8236 ± 0.0099
Validation F1 Score: 0.8242
Confusion Matrix:
[[557   1  43   5]
 [  2  64  19   4]
 [ 70  19 197   9]
 [  5   2   1  26]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90       606
         1.0       0.74      0.72      0.73        89
         2.0       0.76      0.67      0.71       295
         3.0       0.59      0.76      0.67        34

    accuracy                           0.82      1024
   macro avg       0.74      0.77      0.75      1024
weighted avg       0.82      0.82      0.82      1024



```
Performing cross-validation...
Cross-Validation F1 Scores: [0.83028083 0.82905983 0.82051282 0.80562347 0.83251834]
Mean CV F1 Score: 0.8236 ± 0.0099
Validation F1 Score: 0.8242
Confusion Matrix:
[[557   1  43   5]
 [  2  64  19   4]
 [ 70  19 197   9]
 [  5   2   1  26]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90       606
         1.0       0.74      0.72      0.73        89
         2.0       0.76      0.67      0.71       295
         3.0       0.59      0.76      0.67        34

    accuracy                           0.82      1024
   macro avg       0.74      0.77      0.75      1024
weighted avg       0.82      0.82      0.82      1024

```

# Model Training (Grid Search) - 80% Train, 20% Validation

In [53]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'classifier__svm__svc__C': [8.5],
    'classifier__gb__histgradientboostingclassifier__learning_rate': [0.1],
    'classifier__gb__histgradientboostingclassifier__max_iter': [100],
    'classifier__xgb__xgbclassifier__max_depth': [10],
    'classifier__xgb__xgbclassifier__learning_rate': [0.2],

}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="f1_micro",
    cv=stratified_kfold,
    verbose=100,
    n_jobs=-1
)

# Perform grid search
print("Starting Grid Search...")
grid_search.fit(X_train, y_train)

# Print best parameters and score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best CV Score: {grid_search.best_score_:.4f}")

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
val_score = f1_score(y_val, y_pred, average="micro")
print(f"Validation F1 Score: {val_score:.4f}")

Starting Grid Search...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'classifier__gb__histgradientboostingclassifier__learning_rate': 0.1, 'classifier__gb__histgradientboostingclassifier__max_iter': 100, 'classifier__svm__svc__C': 8.5, 'classifier__xgb__xgbclassifier__learning_rate': 0.2, 'classifier__xgb__xgbclassifier__max_depth': 10}
Best CV Score: 0.8309
Validation F1 Score: 0.8281


```
Starting Grid Search...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'classifier__gb__histgradientboostingclassifier__learning_rate': 0.1, 'classifier__gb__histgradientboostingclassifier__max_iter': 100, 'classifier__svm__svc__C': 8.5, 'classifier__xgb__xgbclassifier__learning_rate': 0.2, 'classifier__xgb__xgbclassifier__max_depth': 10}
Best CV Score: 0.8309
Validation F1 Score: 0.8281
```

# Model Evaluation - 100% Train, Test

In [56]:
# Apply SMOTE to the entire dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)  # 100% dataset

best_param = grid_search.best_params_
print(best_param)
# Train final model with best parameters
pipeline.set_params(**best_param)

# Perform cross-validation on the full dataset
cv_scores_final = cross_val_score(pipeline, X, y, cv=stratified_kfold, scoring="f1_micro", n_jobs=-1)
print(f"Final CV F1 Scores: {cv_scores_final.mean():.4f} ± {cv_scores_final.std():.4f}")

# Train the model on the entire resampled (SMOTE) dataset
pipeline.fit(X_resampled, y_resampled)

# Predict on the test set
X_test = test_features.fillna(0).drop(columns=["id"])
y_test_pred = pipeline.predict(X_test)

# Save predictions
submission = pd.DataFrame({
    "id": test_features["id"],
    "y": y_test_pred
})
submission.to_csv("./out/submission_final2_SVM_GB_XGB_Voting_SMOTE_GRIDSEARCH.csv", index=False)
print("Submission file created!")


{'classifier__gb__histgradientboostingclassifier__learning_rate': 0.1, 'classifier__gb__histgradientboostingclassifier__max_iter': 100, 'classifier__svm__svc__C': 8.5, 'classifier__xgb__xgbclassifier__learning_rate': 0.2, 'classifier__xgb__xgbclassifier__max_depth': 10}
Final CV F1 Scores: 0.8310 ± 0.0064
Submission file created!
